In [ ]:
import sys
sys.path.append("..")

In [ ]:
from emeval.input.spec_details import FileSpecDetails

sd = FileSpecDetails("data/", "shankari@eecs.berkeley.edu", "train_bus_ebike_mtv_ucb")

In [ ]:
from emeval.input.phone_view import PhoneView

pv = PhoneView(sd)

In [ ]:
import math

df_map = {}
primary_key = ["spec_id", "phone_os", "phone_label", "start_ts", "end_ts"]

for phone_os, phone_map in pv.map().items():
    for phone_label, phone_detail_map in phone_map.items():
        for ranges in [phone_detail_map["evaluation_ranges"], phone_detail_map["calibration_ranges"]]:
            for r in ranges:
                for key in [k for k in r.keys() if "_df" in k]:
                    # append "primary key" columns to df
                    row_vals = [
                        sd.CURR_SPEC_ID,
                        phone_os,
                        phone_label,
                        math.floor(r["start_ts"]),
                        math.ceil(r["end_ts"])]
                    
                    df = r[key].assign(**dict(zip(primary_key, row_vals)))
                    
                    # move "primary key" columns to front of df
                    df = df[primary_key + [c for c in df.columns.values if c not in primary_key]]
                    
                    # append rows to larger version of df
                    if key not in df_map:
                        df_map[key] = df
                    else:
                        df_map[key] = df_map[key].append(df, ignore_index=True)

In [ ]:
from IPython.display import display

# show fully flattened phone view data
for name, df in df_map.items():
    print(name)
    display(df)